In [4]:
import requests



In [5]:
site = "2710924" #Standort ID
api_key = "1KP4K9CZOFOS9XZU3ZXHWM24RJTXCWAL"


In [33]:
#current PowerFlow
urlpowerflow = "https://monitoringapi.solaredge.com/site/" + site + "/currentPowerFlow?api_key=" + api_key
power = requests.get(urlpowerflow)
if power.status_code == 200:
    print(power.text)

{"siteCurrentPowerFlow":{"updateRefreshRate":3,"unit":"kW","connections":[{"from":"LOAD","to":"Grid"},{"from":"PV","to":"Load"}],"GRID":{"status":"Active","currentPower":6.43},"LOAD":{"status":"Active","currentPower":0.57},"PV":{"status":"Active","currentPower":7.0},"STORAGE":{"status":"Idle","currentPower":0.0,"chargeLevel":100,"critical":false}}}


In [29]:
#Variablen festlegen

'UTF-8'

In [41]:
#generated power
startdate = "2022-04-08"
enddate = "2022-02-15"

urlgenpower = "https://monitoringapi.solaredge.com/site/" + site + "/energy?timeUnit=DAY&endDate=" + enddate + "&startDate=" + startdate + "&api_key=" + api_key

genpower = requests.get(urlgenpower)
if genpower.status_code == 200:
    print (genpower.text)

{"energy":{"timeUnit":"DAY","unit":"Wh","measuredBy":"INVERTER","values":[]}}
